In [ ]:
# Created by Ian Cox | 2/27/2023
# AHEAD | March Madness 2023 Bracket Challenge
# Tournament Prediction | Combining all dataprep functions from the previous scripts for ease
# updating for the 2023 data from kaggle

#kaggle data: 
#      'MNCAATourneyCompactResults.csv'
#      'MNCAATourneySeeds.csv'
#      'MRegularSeasonDetailedResults.csv'

# derived data: 
#      'tourney_outcomes.csv'
#      'team_yearly_points.csv'
#      'tourney_ml_output.csv'

In [1]:
# import libs and set working directory
import pandas as pd
import os
import glob
os.chdir('C:\\Users\\IanCox\\OneDrive - AHEAD\\Documents\\python\\march_madness\\march-machine-learning-mania-2023')

In [2]:
# this function takes 2 dataframes and creates a file with seed info and outcome
def seed_plus_outcome(tourney_results, tourney_seeds):
    # merge the two datasets based on the specified conditions to add seed to the winning team
    merged1 = pd.merge(tourney_results, tourney_seeds[['Seed', 'TeamID', 'Season']], 
                         left_on=['WTeamID', 'Season'], 
                         right_on=['TeamID', 'Season'], 
                         how='left')
    
    # cleanup of the merged dataset
    merge2 = merged1.drop(columns=['TeamID'])
    merge3 = merge2.rename(columns={"Seed": "WSeed"})
    
    # merge the two datasets based on the specified conditions to add seed to the losing team
    merge4 = pd.merge(merge3, seed[['Seed', 'TeamID', 'Season']], 
                         left_on=['LTeamID', 'Season'], 
                         right_on=['TeamID', 'Season'], 
                         how='left')
    
    # cleanup of the merged dataset
    merge5 = merge4.rename(columns={"Seed":"LSeed"})
    seed_merge = merge5.drop(columns=['TeamID'])
    
    # calculate the winning and losing stats
    df_win = seed_merge[['Season', 'DayNum', 'WTeamID', 'WSeed', 'WScore', 'LTeamID', 'LSeed', 'LScore']]
    df_win.columns = ['Season', 'DayNum', 'TeamID', 'TeamSeed', 'Score', 'OpponentID', 'OpponentSeed', 'OpponentScore']
    
    df_loss = seed_merge[['Season', 'DayNum', 'LTeamID', 'LSeed', 'LScore', 'WTeamID', 'WSeed', 'WScore']]
    df_loss.columns = ['Season', 'DayNum', 'TeamID', 'TeamSeed', 'Score', 'OpponentID', 'OpponentSeed', 'OpponentScore']
    
    # combine the win/loss dfs
    df_teams = pd.concat([df_win, df_loss], axis=0, sort=False)
    
    # define a function to compute the outcome (target)
    def compute_outcome(row):
        if row['Score'] > row['OpponentScore']:
            return 1
        else:
            return 0

    # apply the function to each row and create the new target 'outcome' column
    df_teams['outcome'] = df_teams.apply(compute_outcome, axis=1)
    
    #export this df as it is formatted correctly for ml
    df_teams.to_csv('tourney_outcomes.csv',index=False)
    
    return df_teams
    

In [3]:
# define the compart tourney results dataframe and tourney seeds dataframe
tr = pd.read_csv('MNCAATourneyCompactResults.csv')
seed = pd.read_csv('MNCAATourneySeeds.csv')

# use our function to combine this data
seed_plus_outcome(tr, seed)

,Season,DayNum,TeamID,TeamSeed,Score,OpponentID,OpponentSeed,OpponentScore,outcome
0,1985,136,1116,X09,63,1234,X08,54,1
1,1985,136,1120,Z11,59,1345,Z06,58,1
2,1985,136,1207,W01,68,1250,W16,43,1
3,1985,136,1229,Y09,58,1425,Y08,55,1
4,1985,136,1242,Z03,49,1325,Z14,38,1
...,...,...,...,...,...,...,...,...,...
2379,2022,146,1274,Y10,50,1242,Y01,76,0
2380,2022,146,1389,W15,49,1314,W08,69,0
2381,2022,152,1437,Z02,65,1242,Y01,81,0
2382,2022,152,1181,X02,77,1314,W08,81,0


In [13]:
# this function will create aggregate stats for each team for a given year, and export that data as a csv
def team_season_points(year, reg_season_results):
    df = reg_season_results
    df = df[df.Season == year]
    
    points = df.groupby('WTeamID')['WScore'].sum() + df.groupby('LTeamID')['LScore'].sum()
    games = df.groupby('WTeamID').size() + df.groupby('LTeamID').size()
    avg_points = points / games
    
    
    result = pd.concat([points, avg_points], axis=1, keys=['Total Points', 'Average Points']).sort_values(by='Total Points', ascending=False)
    result = result.rename_axis('TeamID').reset_index()
    result['year'] = year
    result = result[['TeamID', 'Total Points', 'Average Points', 'year']]
    fstring = str('team_season_points_%s.csv' % year)
    result.to_csv(fstring, index=False)
    
    return result

In [15]:
# loop through each season, and create a df and csv for each season
regseason = pd.read_csv('MRegularSeasonDetailedResults.csv')
for year in regseason.Season.unique():
    team_season_points(year, regseason)

In [20]:
# function to create an aggreagated season stats dataframe for each team for every year
def create_detailed_agg_file():

    # get all the csv files in the directory
    all_files = glob.glob('team_season_points_*.csv')
    # print statement for sanity check that files were read in correctly
    print(all_files)

    # create an empty list to store dataframes
    dfs = []

    # loop through all the files and read them into dataframes
    for file in all_files:
        df = pd.read_csv(file)
        dfs.append(df)

    # concatenate all the dataframes into a single dataframe
    combined_df = pd.concat(dfs, ignore_index=True)

    # export the seasonal aggregate points into a single csv
    combined_df.to_csv('team_yearly_points.csv',index=False)

In [22]:
# read in all the yearly stats dataframes and concat all of them into one big file for aggregated yearly results by team
create_detailed_agg_file()

['team_season_points_2003.csv', 'team_season_points_2004.csv', 'team_season_points_2005.csv', 'team_season_points_2006.csv', 'team_season_points_2007.csv', 'team_season_points_2008.csv', 'team_season_points_2009.csv', 'team_season_points_2010.csv', 'team_season_points_2011.csv', 'team_season_points_2012.csv', 'team_season_points_2013.csv', 'team_season_points_2014.csv', 'team_season_points_2015.csv', 'team_season_points_2016.csv', 'team_season_points_2017.csv', 'team_season_points_2018.csv', 'team_season_points_2019.csv', 'team_season_points_2020.csv', 'team_season_points_2021.csv', 'team_season_points_2022.csv', 'team_season_points_2023.csv']


In [28]:
# this function combines the tourney_outcomes.csv and team_yearly_points.csv outputs, and combines them into a single df
def generate_ml_input(t_outcomes, year_points):
    # assign the dataframes to the function vars
    to = t_outcomes
    regseason = year_points
    
    # remove the datapoints that are not known in advance of the game
    to_lean = to.drop(columns=['Score','OpponentScore'])

    # merge the dataframes on teamid and year
    merged_data1 = pd.merge(to_lean, regseason, how='left', left_on=['TeamID', 'Season'], right_on=['TeamID', 'year'])
    
    # rename and drop columns
    merge2 = merged_data1.rename(columns={"Total Points":"RS Point Total","Average Points":"RS Avg Points"})
    merge2 = merge2.drop(columns=['year'])
    
    # now do the same for the opponent regular season stats
    merge3 = pd.merge(merge2, regseason, how='left', left_on=['OpponentID', 'Season'], right_on=['TeamID', 'year'])
    
    # rename and drop columns
    merge4 = merge3.rename(columns={"Total Points":"OPP RS Point Total","Average Points":"OPP RS Avg Points"})
    merge5 = merge4.drop(columns=['TeamID_y','year'])
    merge6 = merge5.rename(columns={"TeamID_x":"TeamID"})
    
    # drop rows with missing data
    merge7 = merge6.dropna()
    
    # add a seed difference kpi
    merge7['TeamSeedN'] = merge7['TeamSeed'].str.replace('[A-Za-z]', '')
    merge7['OpponentSeedN'] = merge7['OpponentSeed'].str.replace('[A-Za-z]', '')

    merge7['TeamSeedN'] = pd.to_numeric(merge7['TeamSeedN'])
    merge7['OpponentSeedN'] = pd.to_numeric(merge7['OpponentSeedN'])

    merge7['SeedDiff'] = merge7['TeamSeedN'] - merge7['OpponentSeedN']
    
    # drop the non numeric versions of seed, keep only the ratio var we created
    tourney_ml_output = merge7.drop(columns=['TeamSeed','OpponentSeed','TeamSeedN','OpponentSeedN'])
    
    # write df to file so the DP function don't have to be run again
    tourney_ml_output.to_csv('tourney_ml_input.csv',index=False)

In [29]:
# define the dfs to run through our ml data generation function, we now have our same ml input file with updated 2023 data
to = pd.read_csv('tourney_outcomes.csv')
yp = pd.read_csv('team_yearly_points.csv')

generate_ml_input(to,yp)

C:\Users\IanCox\AppData\Local\Temp\ipykernel_26448\332337799.py:29: FutureWarning: The default value of regex will change from True to False in a future version.
  merge7['TeamSeedN'] = merge7['TeamSeed'].str.replace('[A-Za-z]', '')
C:\Users\IanCox\AppData\Local\Temp\ipykernel_26448\332337799.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merge7['TeamSeedN'] = merge7['TeamSeed'].str.replace('[A-Za-z]', '')
C:\Users\IanCox\AppData\Local\Temp\ipykernel_26448\332337799.py:30: FutureWarning: The default value of regex will change from True to False in a future version.
  merge7['OpponentSeedN'] = merge7['OpponentSeed'].str.replace('[A-Za-z]', '')
C:\Users\IanCox\AppData\Local\Temp\ipykernel_26448\332337799.py:30: SettingWithCopyWarning: 
A valu

## Summary

* All of the code in this notebook is pulled from DP1, DP2, and DP3.

* Individual cell runs from the other notebooks have all been combined to keep things simple.

* These functions can be used to organize the data files provided by kaggle and generate a cleaned up csv (with no leakage and target variable created). 

* You could use any year of the kaggle data so long as the headers match up. I tested these functions with both 2022 and 2023 competition data.

## Ongoing Usage

* The data files for the 2023 competition are still being updated! Once the regular season is over and the seeds are announced on March 12th, 2023, these files will be finalized. We can use the functions in this notebook to quickly generate a new file for ML with the updated data.

* In addition to grabbing the updated files from kaggle, we can also use these functions to generate the correct layout to score data with our model (make predictions). Some of the functions will need to be updated to account for not having the target column in the data for prediction.